[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/ddPn08's_tensor-rt.ipynb)

In [ ]:
# https://github.com/ddPn08/tensorrt-diffusion-colab modified
import os

!apt update && apt install software-properties-common -y && add-apt-repository --yes ppa:deadsnakes/ppa
!apt update && apt install tensorrt tensorrt-dev tensorrt-libs git-lfs -y
!pip install tensorrt==8.5.3.1

In [ ]:
!git clone --single-branch https://github.com/camenduru/TensorRT && \
    cd TensorRT && \
    git submodule update --init --recursive

!cd TensorRT && mkdir -p build && \
    cd build && \
    cmake /content/TensorRT -DTRT_OUT_DIR=$PWD/out && \
    cd plugin && \
    make -j$(nproc)

In [ ]:
from huggingface_hub import create_repo, upload_folder
hf_token = 'token_here' # @param{type:"string"}
repo_id = 'camenduru/libnvinfer_plugin.so.8.5.3_colab' # @param{type:"string"}
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path='/content/TensorRT/build/out', path_in_repo='', repo_id=repo_id, commit_message='content', token=hf_token)

In [ ]:
TRT_OSSPATH = os.path.abspath("./TensorRT")
script_filepath = os.path.join(TRT_OSSPATH, "demo", "Diffusion", "build.py")
!curl -o {script_filepath} -L https://raw.githubusercontent.com/camenduru/tensorrt-diffusion-colab/main/scripts/build.py
!pip install --upgrade pip
!cd TensorRT/demo/Diffusion && pip install -r requirements.txt && pip install numpy==1.21.6

In [ ]:
!git clone https://huggingface.co/trrt/anything-v4.5-vae-swapped /content/model

In [ ]:
unet_pretrained_model_id = "ckpt/anything-v4.5-vae-swapped"  # @param{type:"string"}
vae_pretrained_model_id = "ckpt/anything-v4.5-vae-swapped"  # @param{type:"string"}
clip_pretrained_model_id = "openai/clip-vit-large-patch14"  # @param{type:"string"}

os.environ["UNET_PRETRAINED_MODEL_ID"] = unet_pretrained_model_id
os.environ["VAE_PRETRAINED_MODEL_ID"] = vae_pretrained_model_id
os.environ["CLIP_PRETRAINED_MODEL_ID"] = clip_pretrained_model_id

engine_dir = "/content/model/engine"  # @param{type:"string"}
onnx_dir = "/content/model/onnx"  # @param{type:"string"}

os.makedirs(engine_dir, exist_ok=True)
os.makedirs(onnx_dir, exist_ok=True)

hf_token = ""  # @param{type:"string"}
denoising_prec = "fp32"  # @param ["fp32", "fp16"]
scheduler = "LMSD"  # @param ["LMSD", "DPM"]
height = 512  # @param{type:"slider", min:256, max:1024, step:64}
width = 512  # @param{type:"slider", min:256, max:1024, step:64}

onnx_opset = 16  # @param{type:"integer"}
force_onnx_export = False  # @param{type:"boolean"}
force_onnx_optimize = False  # @param{type:"boolean"}
onnx_minimal_optimization = False  # @param{type:"boolean"}

force_engine_build = False  # @param{type:"boolean"}
build_static_batch = False  # @param{type:"boolean"}
build_dynamic_shape = False  # @param{type:"boolean"}
build_preview_features = False  # @param{type:"boolean"}

def make_args(d):
    arguments = []
    for k, v in d.items():
        k = k.replace("_", "-")
        if type(v) == bool:
            arguments.append(f"--{k}" if v else "")
        elif type(v) == str and v:
            arguments.extend([f"--{k}", f"{v}"])
        elif v:
            arguments.extend([f"--{k}", f"{v}"])
    return " ".join(arguments)

args = make_args({
    "engine-dir": engine_dir,
    "onnx-dir": onnx_dir,
    "hf-token": hf_token,
    "denoising-prec": denoising_prec,
    "scheduler": scheduler,
    "height": height,
    "width": width,
    "onnx-opset": onnx_opset,
    "force-onnx-export": force_onnx_export,
    "force-onnx-optimize": force_onnx_optimize,
    "onnx-minimal-optimization": onnx_minimal_optimization,
    "force-engine-build": force_engine_build,
    "build-static-batch": build_static_batch,
    "build-dynamic-shape": build_dynamic_shape,
    "build-preview-features": build_preview_features
})

! cd /content/TensorRT/demo/Diffusion && LD_PRELOAD="/content/TensorRT/build/out/libnvinfer_plugin.so.8" python {script_filepath} {args}

In [ ]:
from huggingface_hub import create_repo, upload_folder
hf_token = 'token_here' # @param{type:"string"}
repo_id = 'trrt/anything-v4.5-vae-swapped' # @param{type:"string"}
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path='/content/model', path_in_repo='', repo_id=repo_id, commit_message='content', token=hf_token)

In [ ]:
engine_dir = "/content/model/engine"  # @param{type:"string"}
output_dir = "/content/outputs"  # @param{type:"string"}

os.makedirs(output_dir, exist_ok=True)

prompt = "masterpiece, best quality, 1girl"  # @param{type:"string"}
negative_prompt = ""  # @param{type:"string"}
repeat_prompt = 1  # @param [1, 2, 4, 8, 16] {type:"raw"}
height = 512  # @param{type:"slider", min:256, max:1024, step:64}
width = 512  # @param{type:"slider", min:256, max:1024, step:64}
denoising_steps = 50  # @param{type:"integer"}
denoising_prec = "fp32"  # @param ["fp32", "fp16"]
scheduler = "LMSD"  # @param ["LMSD", "DPM"]
seed = "1"  # @param{type:"string"}

num_warmup_runs = 0  # @param{type:"integer"}

def make_args(d):
    arguments = []
    for k, v in d.items():
        k = k.replace("_", "-")
        if type(v) == bool:
            arguments.append(f"--{k}" if v else "")
        elif type(v) == str and v:
            arguments.extend([f"--{k}", f"{v}"])
        elif v:
            arguments.extend([f"--{k}", f"{v}"])
    return " ".join(arguments)

args = make_args({
    "engine-dir": engine_dir,
    "output-dir": output_dir,
    "negative-prompt": negative_prompt,
    "repeat-prompt": repeat_prompt,
    "height": height,
    "width": height,
    "denoising-steps": denoising_steps,
    "denoising-prec": denoising_prec,
    "scheduler": scheduler,
    "seed": seed,
    "num-warmup-runs": num_warmup_runs
})

!cd /content/TensorRT/demo/Diffusion && LD_PRELOAD="/content/TensorRT/build/out/libnvinfer_plugin.so.8" python demo-diffusion.py "{prompt}" {args}

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
!ngrok config add-authtoken token_here

In [ ]:
!pip install colab-xterm

In [ ]:
%load_ext colabxterm
%xterm

In [ ]:
# %env CUDA_MODULE_LOADING=LAZY

In [ ]:
!npm install -g pnpm

In [ ]:
!git clone https://github.com/camenduru/Lsmith.git

In [ ]:
!cd /content/Lsmith/frontend && pnpm i && pnpm build --out-dir ../dist

In [ ]:
!LD_PRELOAD=${PLUGIN_LIBS} python -m uvicorn modules.main:app --host 0.0.0.0 --port 7860 